In [23]:
from matplotlib import pyplot
import json
import datetime, time
import numpy as np
import statsmodels.api as sm

f = open('tweet_data/tweets_#gopatriots.txt')
line = f.readline()
timestamps = []
tweets = []
print ("loading tweets")

loading tweets


In [24]:
while len(line)!=0:
    tweet = json.loads(line)
    tweets.append(tweet)
    timestamps.append(tweet['firstpost_date'])
    line = f.readline()

num_tweets = len(tweets)
print ("tweets loaded")

tweets loaded


In [6]:
#pyplot.hist(timestamps, range=(timestamps[0],timestamps[-1]), bins= 1000)
#pyplot.show()

start_date = datetime.datetime(2015,01,30, 0,0,0)
end_date = datetime.datetime(2015,02,03, 0,0,0)
mintime = int(time.mktime(start_date.timetuple()))
maxtime = int(time.mktime(end_date.timetuple()))
date_reference = datetime.datetime(2015,01,01, 0,0,0)
time_reference = int(time.mktime(date_reference.timetuple()))

print ("extracting features")

num_hours = int((maxtime-mintime)/3600)
num_tweets_in_hour = [0]*num_hours			# number of tweets in the hour
num_retweets_in_hour = [0]*num_hours		# number of retweets in the hour
sum_followers = [0]*num_hours				# sum of the number of followers of users posting tweets in the hour
max_followers = [0]*num_hours				# max number of followers for the users posting tweets in the hour

time_of_day = [0]*num_hours
mintime_daytime = int((mintime-time_reference)/3600) % 24
for t in range(0,num_hours):
    time_of_day[t] = mintime_daytime + t

extracting features


In [25]:
from pprint import pprint
for i in range(100):
    tweet = tweets[i]
    pprint(tweet['tweet']['user']['location'])
#pprint(tweet)

u'Boston, Massachusetts'
u''
u''
u'new york city.'
u'Plantation, FL'
u'Dol Guldur.'
u'Plantation, FL'
u'Boston, Massachusetts'
u'Adelaide, South Australia'
u'Powell, Ohio'
u''
u'Liverpool, ah no, Chitr\xe9 D: '
u'On To The AFC Title Game '
u'Kawasaki,Kanagawa,Japan'
u'Kawasaki,Kanagawa,Japan'
u'Massachusetts'
u''
u'The Universe '
u'New York, NY'
u'North of Boston'
u''
u'Boston, MA'
u'Kawasaki,Kanagawa,Japan'
u'Orlando, FL'
u''
u''
u'Longview, Tx'
u"22 \u2022 enfp \u2022 \u264a\u2022 bi \u2022 5'11"
u'Toluca, M\xe9xico'
u'Patriot Nation'
u'Hurricane, West Virginia'
u''
u''
u'Driver, Seat '
u'CT'
u''
u'New York / Patriots Nation '
u'540'
u'Powell, Ohio'
u''
u'Sheffield'
u'Boston, Massachusetts'
u''
u''
u'Boston, MA'
u'North Carolina '
u'N. Ky./Cincy'
u'Virginia Beach'
u'Cancun, Quintana Roo'
u'Boston, Massachusetts'
u'Boston, Massachusetts'
u'Charlotte, NC'
u'State of Live Free or Die'
u''
u'Powell, Ohio'
u'Mexico'
u'Boston, Massachusetts'
u'New York / Patriots Nation '
u'Boston '
u'Otta

In [ ]:
for i in range(0,num_tweets):
    tweet = tweets[i]
    tweet_time = tweet['firstpost_date']
    if tweet_time >= mintime:
        hour = int((tweet_time-mintime)/3600)
        if hour >= num_hours:
            break;
        num_tweets_in_hour[hour] += 1
        try:
            num_retweets_in_hour += tweet['metrics']['citations']['data'][0]['citations']
        except:
            pass
        sum_followers[hour] += tweet['tweet']['user']['followers_count']
        max_followers[hour] = max(max_followers[hour], tweet['tweet']['user']['followers_count'])

print ("fitting the model")

y = num_tweets_in_hour[1:]
x = [num_tweets_in_hour[0:-1] , num_retweets_in_hour[0:-1] , sum_followers[0:-1] , max_followers[0:-1] , time_of_day[0:-1]]

def reg_m(y, x):
    ones = np.ones(len(x[0]))
    X = sm.add_constant(np.column_stack((x[0], ones)))
    for ele in x[1:]:
        X = sm.add_constant(np.column_stack((ele, X)))
#    results = sm.OLS(y, X).fit()
    results = sm.regression.linear_model.OLS(y, X).fit()
    return results

model = reg_m(y, x)
var_names = model.model.data.xnames
const_index = var_names.index('const')
x_p = [1]*len(x)
x_p.insert(const_index, 0)
pr = model.predict(x_p)
print model.summary()